In [2]:
from typing import Set, List, Dict, Tuple, Callable
from dataclasses import dataclass
import numpy as np
import bisect 

INFINITY = 1e700

@dataclass
class Queue(object):
    _elements: List[Tuple[int, Dict[int, float], int]]

    def __init__(self, Gv: Dict[int, Dict[int, float]], Tv:Dict[int, int]):
        self._elements = []
        for v_i in Gv.keys():
            self.enqueue([Tv[v_i], Gv[v_i], v_i])

    def dequeue(self):
        _head = self._elements[0]
        self._elements.pop(0)
        return _head
    def head(self):
        return self._elements[0]
    
    def update(self, element: Tuple[int, Dict[int, float], int]):
        for _element in self._elements:
            if _element[2] == element[2]:
                _element[0:2] = element[0:2]
        

    def enqueue(self, element: Tuple[int, Dict[int, float], int]):
        bisect.insort_right(self._elements, element, key=lambda e: e[1][e[0]])
        
        
    def size(self):
        return len(self._elements)


class TDG:
    V: Set[int]
    E: Set[Tuple[int, int, List[float]]]
    W: Callable[[Tuple[int, int, List[float]], float], float]
    
    def __init__(self, len_v: int, E: Set[Tuple[int, int, List[float]]]):
        self.V = set(range(len_v))
        self.E = E
        # TODO: definir un W par défaut...

@dataclass
class TwoStepLTTSolver:
    tdg: TDG
    v_s: int
    v_e: int
    T: Tuple[int, int]

    def two_step_ltt(self) -> Tuple[int, Tuple[int, int, List[float]]]:
        Gv = self.time_refinement(self.tdg, self.v_s, self.v_e, self.T)
        t_star = self.T[0] + np.argmin([Gv[self.v_e][t] for t in range(self.T[0], self.T[1]+1)])
        if Gv[self.v_e][t_star] == INFINITY:
            return (-1, [])
        p_star = self.path_selection(Gv, t_star)
        return t_star, p_star

    def time_refinement(self) -> Dict[int, Dict[int, float]]:
        Gv = dict()
        Tv = dict()
        for v_i in self.tdg.V:
            Tv[v_i] = t 
            Gv[v_i] = dict()
            for t in range(self.T[0], self.T[1]+1):
                Gv[v_i][t] = t if v_i == self.v_s else INFINITY
            Q = Queue(Gv, Tv)
            while Q.size() >= 2:
                t_i, g_i, v_i = Q.dequeue()
                t_k, g_k, v_k = Q.head()
                delta = min(self.tdg.W(edge, g_k[t_k]) if edge[1] == v_i else INFINITY for edge in self.tdg.E)
                t_prime_i = max(t if g_i[t] <= g_k[t] + delta else -INFINITY for t in range(self.T[0], self.T[1]+1))
                for v_ii, v_j, chunks in self.tdg.E:
                    if v_ii != v_i:
                        continue
                    g_prime_j = [g_i[t] + self.tdg.W((v_i, v_j, chunks), t) for t in range(t_i, t_prime_i+1)]
                    g_j = [min(g_j[t], g_prime_j[t]) for t in range(t_i, t_prime_i+1)]
                    Q.update(Tv[v_j], g_j)
                t_i = t_prime_i
                
                if t_i >= Tv[self.v_e]: 
                    if v_i == self.v_e:
                        return Gv
                else:
                    Q.enqueue((t_i, g_i, v_i))
                
            return Gv
                

    def path_selection(self, Gv: Dict[int, Dict[int, float]], t_star: int) -> List[Tuple[int, int, List[float]]]:
        v_j = self.v_e
        p_star = []
        while v_j != self.v_s:
            for edge in self.tdg.E:
                if edge[1] != v_j:
                    continue
                v_i = edge[0]
                if Gv[v_i][t_star] + self.tdg.W(edge, Gv[v_i][t_star]) == Gv[v_j][t_star]:
                    v_j = edge[0]
                    break
            p_star = [edge] + p_star

        return p_star


In [7]:
from typing import Set, List, Dict, Tuple, Callable
from dataclasses import dataclass
import numpy as np
import heapq

INFINITY = 1e700

@dataclass
class TDG:
    V: List[int]
    E: List[Tuple[int, int, List[float]]]
    W: Callable[[Tuple[int, int, List[float]], float], float]
    
    def __init__(self, len_v: int, E: Set[Tuple[int, int, List[float]]], W: Callable[[Tuple[int, int, List[float]], float], float]):
        self.V = set(range(len_v))
        self.E = E
        self.W = W

@dataclass
class TwoStepLTTSolver:
    tdg: TDG
    v_s: int
    v_e: int
    T: Tuple[float, float]

    def two_step_ltt(self) -> Tuple[float, List[Tuple[int, int, List[float]]]]:
        Gv = self.time_refinement()
        if all(Gv[self.v_e][t] == INFINITY for t in range(int(self.T[0]), int(self.T[1])+1)):
            return (-1, [])
        
        t_values = [t for t in range(int(self.T[0]), int(self.T[1])+1) if Gv[self.v_e][t] < INFINITY]
        if not t_values:
            return (-1, [])
            
        t_star = min(t_values, key=lambda t: Gv[self.v_e][t] - t)
        p_star = self.path_selection(Gv, t_star)
        return (t_star, p_star)

    def time_refinement(self) -> Dict[int, Dict[float, float]]:
        Gv = dict()
        tau = dict()
        
        # Initialisation
        for v_i in self.tdg.V:
            Gv[v_i] = dict()
            if v_i == self.v_s:
                for t in range(int(self.T[0]), int(self.T[1])+1):
                    Gv[v_i][t] = float(t)
                tau[v_i] = self.T[0]
            else:
                for t in range(int(self.T[0]), int(self.T[1])+1):
                    Gv[v_i][t] = INFINITY
                tau[v_i] = self.T[0]
        
        # File de priorité: (g_i(tau_i), tau_i, v_i)
        Q = []
        for v_i in self.tdg.V:
            heapq.heappush(Q, (Gv[v_i][tau[v_i]], tau[v_i], v_i))
        
        while len(Q) >= 2:
            g_i_tau_i, tau_i, v_i = heapq.heappop(Q)
            
            if len(Q) == 0:
                break
            
            g_k_tau_k, tau_k, v_k = Q[0]
            
            # Calcul de Delta
            delta = INFINITY
            for edge in self.tdg.E:
                if edge[1] == v_i:
                    v_f = edge[0]
                    w_fi = self.tdg.W(edge, g_k_tau_k)
                    delta = min(delta, w_fi)
            
            # Calcul de tau_prime_i
            tau_prime_i = tau_i
            for t in range(int(tau_i), int(self.T[1])+1):
                if Gv[v_i][t] <= g_k_tau_k + delta:
                    tau_prime_i = float(t)
                else:
                    break
            
            # Mise à jour des voisins
            for edge in self.tdg.E:
                if edge[0] == v_i:
                    v_j = edge[1]
                    for t in range(int(tau_i), int(tau_prime_i)+1):
                        if Gv[v_i][t] < INFINITY:
                            g_prime_j = Gv[v_i][t] + self.tdg.W(edge, Gv[v_i][t])
                            Gv[v_j][t] = min(Gv[v_j][t], g_prime_j)
                    
                    # Mise à jour de tau[v_j] si nécessaire
                    new_tau_j = tau[v_j]
                    for t in range(int(tau[v_j]), int(tau_prime_i)+1):
                        if Gv[v_j][t] < INFINITY:
                            new_tau_j = float(t)
                            break
                    
                    # Retirer l'ancien élément de Q et ajouter le nouveau
                    Q = [(g, t, v) for (g, t, v) in Q if v != v_j]
                    heapq.heapify(Q)
                    if new_tau_j <= self.T[1]:
                        heapq.heappush(Q, (Gv[v_j][new_tau_j], new_tau_j, v_j))
                    tau[v_j] = new_tau_j
            
            tau[v_i] = tau_prime_i
            
            if tau_prime_i >= self.T[1]:
                if v_i == self.v_e:
                    return Gv
            else:
                heapq.heappush(Q, (Gv[v_i][tau_prime_i], tau_prime_i, v_i))
        
        return Gv

    def path_selection(self, Gv: Dict[int, Dict[float, float]], t_star: float) -> List[Tuple[int, int, List[float]]]:
        v_j = self.v_e
        p_star = []
        
        while v_j != self.v_s:
            found = False
            for edge in self.tdg.E:
                if edge[1] == v_j:
                    v_i = edge[0]
                    if (t_star in Gv[v_i] and 
                        Gv[v_i][t_star] < INFINITY and 
                        abs(Gv[v_i][t_star] + self.tdg.W(edge, Gv[v_i][t_star]) - Gv[v_j][t_star]) < 1e-9):
                        p_star = [edge] + p_star
                        v_j = v_i
                        found = True
                        break
            
            if not found:
                # Cas où on ne trouve pas de prédécesseur exact
                # On cherche le meilleur prédécesseur
                best_edge = None
                best_diff = INFINITY
                for edge in self.tdg.E:
                    if edge[1] == v_j:
                        v_i = edge[0]
                        if t_star in Gv[v_i] and Gv[v_i][t_star] < INFINITY:
                            diff = abs(Gv[v_i][t_star] + self.tdg.W(edge, Gv[v_i][t_star]) - Gv[v_j][t_star])
                            if diff < best_diff:
                                best_diff = diff
                                best_edge = edge
                
                if best_edge:
                    p_star = [best_edge] + p_star
                    v_j = best_edge[0]
                else:
                    break
        
        return p_star

In [3]:
# import matplotlib.pyplot as plt
import numpy as np
from typing import Set, List, Dict, Tuple, Callable

# Fonction de poids pour simuler un graphe dépendant du temps
def create_time_dependent_weight_function():
    """
    Crée une fonction de poids qui varie selon le temps de départ
    Simule des conditions de trafic variables (heures de pointe, etc.)
    """
    def W(edge: Tuple[int, int, List[float]], departure_time: float) -> float:
        v_i, v_j, chunks = edge
        base_weight = sum(chunks)  # Poids de base = somme des longueurs des tronçons
        
        # Simulation de conditions de trafic
        # Heures de pointe: 7-9h et 17-19h
        hour = departure_time % 24
        
        if 7 <= hour <= 9 or 17 <= hour <= 19:
            # Trafic dense pendant les heures de pointe
            traffic_factor = 1.5 + 0.3 * np.sin(hour * np.pi / 12)
        elif 0 <= hour <= 6 or 22 <= hour <= 24:
            # Trafic léger la nuit
            traffic_factor = 0.8
        else:
            # Trafic normal
            traffic_factor = 1.0
        
        # Ajouter une variation selon l'arc
        edge_factor = 1.0 + 0.2 * np.sin((v_i + v_j) * np.pi / 5)
        
        return base_weight * traffic_factor * edge_factor
    
    return W

def create_test_graph():
    """
    Crée un graphe de test représentant un petit réseau routier
    """
    # 6 nœuds représentant des intersections
    len_v = 6
    
    # Arcs avec leurs tronçons (longueurs en km)
    # Format: (nœud_source, nœud_destination, [longueurs_tronçons])
    E = [
        [0, 1, [2.0, 1.5]],      # Route directe
        (1, 3, [1.5]),           # Connexion courte
        (0, 2, [1.0, 1.0, 1.0]), # Route alternative plus longue
        (1, 4, [3.0, 2.0]),      # Route longue
        (2, 3, [2.0]),           # Connexion moyenne
        (2, 4, [1.5, 1.0]),      # Route alternative
        (3, 5, [2.5, 1.5]),      # Vers destination
        (4, 5, [1.0, 1.0]),      # Route courte vers destination
        (1, 5, [6.0]),           # Route directe mais longue
    ]
    
    W = create_time_dependent_weight_function()
    
    return TDG(len_v, E, W)

def visualize_graph(tdg: TDG, path: List[Tuple[int, int, List[float]]] = None):
    """
    Visualise le graphe et le chemin optimal si fourni
    """
    plt.figure(figsize=(10, 8))
    
    # Positions des nœuds pour la visualisation
    pos = {
        0: (0, 1),
        1: (1, 2),
        2: (1, 0),
        3: (2, 1.5),
        4: (2, 0.5),
        5: (3, 1)
    }
    
    # Dessiner les arcs
    for edge in tdg.E:
        v_i, v_j, chunks = edge
        x_values = [pos[v_i][0], pos[v_j][0]]
        y_values = [pos[v_i][1], pos[v_j][1]]
        
        # Colorer différemment si l'arc fait partie du chemin optimal
        if path and edge in path:
            plt.plot(x_values, y_values, 'r-', linewidth=3, label='Chemin optimal' if edge == path[0] else '')
        else:
            plt.plot(x_values, y_values, 'b-', alpha=0.5)
        
        # Afficher la longueur totale de l'arc
        mid_x = (x_values[0] + x_values[1]) / 2
        mid_y = (y_values[0] + y_values[1]) / 2
        total_length = sum(chunks)
        plt.text(mid_x, mid_y, f'{total_length:.1f}km', fontsize=8, ha='center')
    
    # Dessiner les nœuds
    for node in tdg.V:
        x, y = pos[node]
        plt.scatter(x, y, s=200, c='lightblue', edgecolors='black', zorder=5)
        plt.text(x, y, str(node), fontsize=12, ha='center', va='center')
    
    plt.xlabel('Position X')
    plt.ylabel('Position Y')
    plt.title('Graphe dépendant du temps avec chemin optimal')
    plt.grid(True, alpha=0.3)
    if path:
        plt.legend()
    plt.tight_layout()
    plt.show()

def analyze_travel_times(tdg: TDG, v_s: int, v_e: int, T: Tuple[float, float]):
    """
    Analyse les temps de trajet pour différents temps de départ
    """
    solver = TwoStepLTTSolver(tdg, v_s, v_e, T)
    
    # Calculer les fonctions d'arrivée
    Gv = solver.time_refinement()
    
    # Analyser les temps de trajet pour différents temps de départ
    departure_times = []
    travel_times = []
    
    for t in range(int(T[0]), int(T[1])+1):
        if Gv[v_e][t] < INFINITY:
            departure_times.append(t)
            travel_times.append(Gv[v_e][t] - t)
    
    if departure_times:
        plt.figure(figsize=(10, 6))
        plt.plot(departure_times, travel_times, 'b-', linewidth=2)
        
        # Marquer le temps de départ optimal
        t_star, p_star = solver.two_step_ltt()
        if t_star != -1:
            optimal_travel_time = Gv[v_e][t_star] - t_star
            plt.scatter([t_star], [optimal_travel_time], c='red', s=100, zorder=5)
            plt.annotate(f'Optimal: départ à {t_star}h\nTemps: {optimal_travel_time:.2f}h', 
                        xy=(t_star, optimal_travel_time), 
                        xytext=(t_star+2, optimal_travel_time+0.5),
                        arrowprops=dict(arrowstyle='->', color='red'))
        
        plt.xlabel('Heure de départ')
        plt.ylabel('Temps de trajet (heures)')
        plt.title(f'Temps de trajet de {v_s} à {v_e} selon l\'heure de départ')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    return departure_times, travel_times

# Programme principal de test
if __name__ == "__main__":
    print("=== Test de l'algorithme Two-Step LTT ===\n")
    
    # Créer le graphe de test
    tdg = create_test_graph()
    print(f"Graphe créé avec {len(tdg.V)} nœuds et {len(tdg.E)} arcs")
    
    # Définir la requête
    v_s = 0  # Source
    v_e = 5  # Destination
    T = (0.0, 24.0)  # Fenêtre de temps (24 heures)
    
    print(f"\nRecherche du meilleur itinéraire de {v_s} à {v_e}")
    print(f"Fenêtre de temps considérée: {T[0]}h à {T[1]}h\n")
    
    # Résoudre le problème
    solver = TwoStepLTTSolver(tdg, v_s, v_e, T)
    t_star, p_star = solver.two_step_ltt()
    
    if t_star != -1:
        print(f"✅ Chemin optimal trouvé!")
        print(f"   - Heure de départ optimale: {t_star}h")
        print(f"   - Chemin: ", end="")
        for i, edge in enumerate(p_star):
            if i == 0:
                print(f"{edge[0]}", end="")
            print(f" → {edge[1]}", end="")
        print()
        
        # Calculer le temps de trajet
        Gv = solver.time_refinement()
        travel_time = Gv[v_e][t_star] - t_star
        print(f"   - Temps de trajet: {travel_time:.2f} heures")
        
        # Calculer la distance totale
        total_distance = sum(sum(edge[2]) for edge in p_star)
        print(f"   - Distance totale: {total_distance:.1f} km")
    else:
        print("❌ Aucun chemin trouvé!")
    
    # Visualisations
    print("\n=== Visualisations ===")
    
    # 1. Visualiser le graphe avec le chemin optimal
    visualize_graph(tdg, p_star if t_star != -1 else None)
    
    # 2. Analyser les temps de trajet selon l'heure de départ
    departure_times, travel_times = analyze_travel_times(tdg, v_s, v_e, T)
    
    # Test avec différentes paires source-destination
    print("\n=== Tests supplémentaires ===")
    test_pairs = [(0, 3), (2, 5), (1, 4)]
    
    for v_s, v_e in test_pairs:
        solver = TwoStepLTTSolver(tdg, v_s, v_e, (0.0, 24.0))
        t_star, p_star = solver.two_step_ltt()
        
        if t_star != -1:
            Gv = solver.time_refinement()
            travel_time = Gv[v_e][t_star] - t_star
            print(f"\n{v_s} → {v_e}: Départ optimal à {t_star}h, temps de trajet: {travel_time:.2f}h")
        else:
            print(f"\n{v_s} → {v_e}: Aucun chemin trouvé")

=== Test de l'algorithme Two-Step LTT ===



TypeError: TDG.__init__() takes 3 positional arguments but 4 were given